In [1]:
import re
import requests
from bs4 import BeautifulSoup
from bs4 import Tag
import pandas as pd
import numpy as np

In [2]:
#Load app list from from other csv
store = pd.read_csv('../dataset/googleplaystore.csv')
appNames = store['App']
len(appNames)

10841

In [3]:
#Dictionary of all app details
allAppsDetail = {}

#Function to find innermost text in recursive tag
def find_innermost_text(input):
    end = input
    while(isinstance(end, Tag)):
        try:
            end = end.contents[0]
        except:
            return np.NaN
    return end

#Iterate over all the apps
for app in appNames[0:300]:
    
    #Dictionary to store app details
    details = {}
    
    # Search the app on playstore and crawl the result
    appWithPlus = app.replace(' ','+')
    html_page = requests.get("https://play.google.com/store/search?q="+appWithPlus+"&c=apps")
    soup = BeautifulSoup(html_page.text,'html.parser')
    
    # Extract the href(package name for the app) 
    applink = soup.find('div',attrs={'title':app})
    if(applink==None):
        continue
    applink = applink.parent.attrs['href']
    
    # Open the app specific page
    url='https://play.google.com'+applink
    data=requests.get(url)
    soup = BeautifulSoup(data.text,'html.parser')
    
    # Extract contents from app specific page
    atr_labels = soup.find_all('div',attrs={'class':'BgcNfc'})
    atr_values = soup.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['IQ1z0d'])
    
    # Fill out app specific details in a dictionary
    for index in range(len(atr_labels)):
        details.update({atr_labels[index].contents[0] : find_innermost_text(atr_values[index])}) 
            
    allAppsDetail.update({app:details})

In [4]:
result_df = pd.DataFrame(allAppsDetail)
#Transpose result df
apps_df = result_df.T.reset_index().rename(columns={'index':'App'})
apps_df.head(5)

,App,Updated,Size,Installs,Current Version,Requires Android,Content Rating,Permissions,Report,Offered By,Developer,Interactive Elements,In-app Products,Eligible for Family Library
0,Pixel Draw - Number Art Coloring Book,"June 27, 2019",3.3M,"1,000,000+",1.4,4.4 and up,Everyone,View details,Flag as inappropriate,KidsAppBox,Visit website,NaN,NaN,NaN
1,Infinite Painter,"November 11, 2019",44M,"5,000,000+",6.3.59,4.2 and up,Everyone,View details,Flag as inappropriate,Infinite Studio Mobile,Visit website,"Users Interact, Digital Purchases",$1.99 - $9.99 per item,NaN
2,Garden Coloring Book,"March 31, 2020",34M,"1,000,000+",3.0.10,4.1 and up,Teen,View details,Flag as inappropriate,Colorfit,colorfit.app@outlook.com,In-App Purchases,$1.99 - $7.16 per item,NaN
3,Name Art Photo Editor - Focus n Filters,"March 22, 2020",11M,"50,000+",12.0,4.1 and up,Everyone,View details,Flag as inappropriate,artsoftech,Visit website,Users Interact,NaN,NaN
4,Tattoo Name On My Photo Editor,"December 30, 2019",23M,"10,000,000+",4.0,4.1 and up,Teen,View details,Flag as inappropriate,iDroid Solution,trendingmobileapps@gmail.com,NaN,NaN,NaN


In [5]:
# Write to csv
#apps_df.to_csv('dataset/apps_additional_info.csv',index=False)